In [1]:
import numpy as np
import pytorch_ranger
from contr_loss import *
from tensorlayer import *
from preprocessing import *
import pytorch_lightning as pl
import torch.nn.functional as F

In [2]:
dm = dna2vecDataModule(
    vector_path="../dna2vec/pretrained/dna2vec-20161219-0153-k3to8-100d-10c-29320Mbp-sliding-Xat.w2v", 
    fasta_path="training_set.fasta",
    kmers=[3,4,5,6,7,8], batch_size=64
)

In [3]:
class TensorSiamese(pl.LightningModule):
    
    def __init__(self, order, input_dim, output_dim, rank_tucker=5):
        super(TensorSiamese, self).__init__()
        self.tensor = NeuralTensorLayer(
            order, input_dim, output_dim, rank_tucker=rank_tucker
        )
        self.bn = nn.BatchNorm1d(output_dim)
        self.s = nn.Sigmoid()
        self.loss = ContrastiveLoss()
        
    def forward(self, x):
        return(self.s(self.bn(self.tensor(x))))
    
    def configure_optimizers(self):
        optimizer = pytorch_ranger.RangerQH(self.parameters())
        return(optimizer)
    
    def training_step(self, train_batch, batch_idx):
        x, y = train_batch
        x1 = x[:,0:100]
        x2 = x[:,100:]
        left = self.forward(x1)
        right = self.forward(x2)
        loss = self.loss(left, right, y)+0.5*self.tensor.get_orthogonality_loss()
        self.log("train_loss", loss)
        return(loss)
    
    def validation_step(self, val_batch, batch_idx):
        x, y = val_batch
        x1 = x[:,0:100]
        x2 = x[:,100:]
        left = self.forward(x1)
        right = self.forward(x2)
        loss = self.loss(left, right, y)+0.5*self.tensor.get_orthogonality_loss()
        self.log("val_loss", loss)
        return(loss)
    
    def predict(self, val_batch):
        x, _ = val_batch
        x1 = x[:,0:100]
        x2 = x[:,100:]
        left = self.forward(x1)
        right = self.forward(x2)
        return(F.pairwise_distance(left,right))

In [4]:
model = TensorSiamese(3, 100, 50).cuda()

/home/bakirillov/HDD/works/Hackathons/genhack2020/src/tensorlayer.py:39: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.
  initializer(self.bias)


In [5]:
trainer = pl.Trainer(
    gpus="0", max_epochs=100,
    default_root_dir="../checkpoints"
)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [ ]:
trainer.fit(model, dm)

/home/bakirillov/anaconda3/envs/lapki/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/bakirillov/anaconda3/envs/lapki/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/bakirillov/anaconda3/envs/lapki/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/bakirillov/anaconda3/envs/lapki

/home/bakirillov/anaconda3/envs/lapki/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:45: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)


/home/bakirillov/anaconda3/envs/lapki/lib/python3.7/site-packages/pytorch_ranger/rangerqh.py:152: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:766.)
  exp_avg.mul_(beta1_adj).add_(1.0 - beta1_adj, d_p)
